<a href="https://colab.research.google.com/github/Goal48/Graindient_Boosting_Classification_Implementation/blob/main/Gradient_Boosting_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Holiday-Package-Prediction**

"Trips & Travel.Com" company wants to enable and establish a viable business model to expand the customer base. One of the ways to expand the customer base is to introduce a new offering of packages. Currently, there are 5 types of packages the company is offering - Basic, Standard, Deluxe, Super Deluxe, King. Looking at the data of the last year, we observed that 18% of the customers purchased the packages. However, the marketing cost was quite high because customers were contacted at random without looking at the available information. The company is now planning to launch a new product i.e. Wellness Tourism Package. Wellness Tourism is defined as Travel that allows the traveler to maintain, enhance or kick-start a healthy lifestyle, and support or increase one's sense of well-being. However, this time company wants to harness the available data of existing and potential customers to make the marketing expenditure more efficient

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import plotly.express as px

In [3]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Holiday Package Prediction.zip')

In [4]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,NaN,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [5]:
df.shape

(4888, 20)


Data Cleaning
1.   Handling Missing Values
2.   Handing Duplicates
3.   Check Data Types
4.   Understand the Datasets








In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4888 entries, 0 to 4887
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   CustomerID                4888 non-null   int64  
 1   ProdTaken                 4888 non-null   int64  
 2   Age                       4662 non-null   float64
 3   TypeofContact             4863 non-null   object 
 4   CityTier                  4888 non-null   int64  
 5   DurationOfPitch           4637 non-null   float64
 6   Occupation                4888 non-null   object 
 7   Gender                    4888 non-null   object 
 8   NumberOfPersonVisiting    4888 non-null   int64  
 9   NumberOfFollowups         4843 non-null   float64
 10  ProductPitched            4888 non-null   object 
 11  PreferredPropertyStar     4862 non-null   float64
 12  MaritalStatus             4888 non-null   object 
 13  NumberOfTrips             4748 non-null   float64
 14  Passport

In [7]:
df.isnull().sum()

,0
CustomerID,0
ProdTaken,0
Age,226
TypeofContact,25
CityTier,0
DurationOfPitch,251
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,45


In [8]:
df['Gender'].value_counts()

,count
Gender,
Male,2916
Female,1817
Fe Male,155


In [9]:
df['Gender'] = df['Gender'].replace('Fe Male', 'Female')

In [10]:
df['Gender'].value_counts()

,count
Gender,
Male,2916
Female,1972


In [11]:
df['MaritalStatus'].value_counts()

,count
MaritalStatus,
Married,2340
Divorced,950
Single,916
Unmarried,682


In [12]:
df['MaritalStatus'] = df['MaritalStatus'].replace('Unmarried', 'Single')

In [13]:
df['MaritalStatus'].value_counts()

,count
MaritalStatus,
Married,2340
Single,1598
Divorced,950


In [14]:
df['TypeofContact'].value_counts()

,count
TypeofContact,
Self Enquiry,3444
Company Invited,1419


In [15]:
## Checking Missing Values
## Vaules which have 'nan' values
features_with_na=[features for features in df.columns if df[features].isnull().sum()>=1]
for feature in features_with_na:
    print(feature,np.round(df[feature].isnull().mean()*100,4),'% missing values')

Age 4.6236 % missing values
TypeofContact 0.5115 % missing values
DurationOfPitch 5.135 % missing values
NumberOfFollowups 0.9206 % missing values
PreferredPropertyStar 0.5319 % missing values
NumberOfTrips 2.8642 % missing values
NumberOfChildrenVisiting 1.3502 % missing values
MonthlyIncome 4.7668 % missing values


In [16]:
df[features_with_na].select_dtypes(include=['int64','float64']).describe()

,Age,DurationOfPitch,NumberOfFollowups,PreferredPropertyStar,NumberOfTrips,NumberOfChildrenVisiting,MonthlyIncome
count,4662.000000,4637.000000,4843.000000,4862.000000,4748.000000,4822.000000,4655.000000
mean,37.622265,15.490835,3.708445,3.581037,3.236521,1.187267,23619.853491
std,9.316387,8.519643,1.002509,0.798009,1.849019,0.857861,5380.698361
min,18.000000,5.000000,1.000000,3.000000,1.000000,0.000000,1000.000000
25%,31.000000,9.000000,3.000000,3.000000,2.000000,1.000000,20346.000000
50%,36.000000,13.000000,4.000000,3.000000,3.000000,1.000000,22347.000000
75%,44.000000,20.000000,4.000000,4.000000,4.000000,2.000000,25571.000000
max,61.000000,127.000000,6.000000,5.000000,22.000000,3.000000,98678.000000


Imputing Null values
1. Impute Median value for Age Column
2. Impute Mode value for Type od Contract Column
3. Impute Median value for duration of Pitch Column
4. Impute Mode value for Number Followup as it is Discrete features Column
5. Impute Mode value for PreferredPropertyStar Column
6. Impute Median value for Numberoftrips Column
7. Impute Mode value for NumberOfChildrenVisting Column
8. Impute Median value for MonthlyIncome Column

In [17]:
df['Age'].fillna(df['Age'].median(),inplace=True)
df['TypeofContact'].fillna(df['TypeofContact'].mode()[0],inplace=True)
df['DurationOfPitch'].fillna(df['DurationOfPitch'].median(),inplace=True)
df['NumberOfFollowups'].fillna(df['NumberOfFollowups'].mode()[0],inplace=True)
df['PreferredPropertyStar'].fillna(df['PreferredPropertyStar'].mode()[0],inplace=True)
df['NumberOfTrips'].fillna(df['NumberOfTrips'].median(),inplace=True)
df['NumberOfChildrenVisiting'].fillna(df['NumberOfChildrenVisiting'].mode()[0],inplace=True)
df['MonthlyIncome'].fillna(df['MonthlyIncome'].median(),inplace=True)

In [18]:
df.isnull().sum()

,0
CustomerID,0
ProdTaken,0
Age,0
TypeofContact,0
CityTier,0
DurationOfPitch,0
Occupation,0
Gender,0
NumberOfPersonVisiting,0
NumberOfFollowups,0


**Feature Engineering**

In [19]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfPersonVisiting,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,NumberOfChildrenVisiting,Designation,MonthlyIncome
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3,3.0,Deluxe,3.0,Single,1.0,1,2,1,0.0,Manager,20993.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,3,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,2.0,Manager,20130.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,3,4.0,Basic,3.0,Single,7.0,1,3,0,0.0,Executive,17090.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,2,3.0,Basic,3.0,Divorced,2.0,1,5,1,1.0,Executive,17909.0
4,200004,0,36.0,Self Enquiry,1,8.0,Small Business,Male,2,3.0,Basic,4.0,Divorced,1.0,0,5,1,0.0,Executive,18468.0


In [20]:
df['TotalVisits'] = df['NumberOfChildrenVisiting'] + df['NumberOfPersonVisiting']

In [21]:
df.drop(columns=['NumberOfChildrenVisiting','NumberOfPersonVisiting'],inplace=True)

In [22]:
df.head()

,CustomerID,ProdTaken,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalVisits
0,200000,1,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Single,1.0,1,2,1,Manager,20993.0,3.0
1,200001,0,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,200002,1,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Single,7.0,1,3,0,Executive,17090.0,3.0
3,200003,0,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,200004,0,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [23]:
## get all numerical Features
features_num = [feature for feature in df.columns if df[feature].dtypes!='O']
print('Number of Numerical Features:',len(features_num))

Number of Numerical Features: 13


In [24]:
## get all Catergorical Features
features_cat = [feature for feature in df.columns if df[feature].dtypes=='O']
print('Number of Numerical Features:',len(features_cat))

Number of Numerical Features: 6


In [25]:
## Discrete Features
features_Dis = [feature for feature in features_num if len(df[feature].unique())<25]
print('Number of Discrete Features:',len(features_Dis))

Number of Discrete Features: 9


In [26]:
## Continuos Features
features_con = [feature for feature in features_num if feature not in features_Dis]
print('Number of Continuos Features:',len(features_con))

Number of Continuos Features: 4


Training Test Split And Model Training

In [27]:
from sklearn.model_selection import train_test_split
X = df.drop(columns=['ProdTaken'],axis=1)
y = df['ProdTaken']

In [28]:
y.value_counts()

,count
ProdTaken,
0,3968
1,920


In [29]:
X.head()

,CustomerID,Age,TypeofContact,CityTier,DurationOfPitch,Occupation,Gender,NumberOfFollowups,ProductPitched,PreferredPropertyStar,MaritalStatus,NumberOfTrips,Passport,PitchSatisfactionScore,OwnCar,Designation,MonthlyIncome,TotalVisits
0,200000,41.0,Self Enquiry,3,6.0,Salaried,Female,3.0,Deluxe,3.0,Single,1.0,1,2,1,Manager,20993.0,3.0
1,200001,49.0,Company Invited,1,14.0,Salaried,Male,4.0,Deluxe,4.0,Divorced,2.0,0,3,1,Manager,20130.0,5.0
2,200002,37.0,Self Enquiry,1,8.0,Free Lancer,Male,4.0,Basic,3.0,Single,7.0,1,3,0,Executive,17090.0,3.0
3,200003,33.0,Company Invited,1,9.0,Salaried,Female,3.0,Basic,3.0,Divorced,2.0,1,5,1,Executive,17909.0,3.0
4,200004,36.0,Self Enquiry,1,8.0,Small Business,Male,3.0,Basic,4.0,Divorced,1.0,0,5,1,Executive,18468.0,2.0


In [30]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [31]:
X_train.shape , X_test.shape

((3910, 18), (978, 18))

In [32]:
## Create column Transformer with 3 types of Transformers
features_cat = X.select_dtypes(include='object').columns
features_num = X.select_dtypes(exclude='object').columns

In [33]:
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer = StandardScaler()
oh_transformer = OneHotEncoder()
preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder",oh_transformer,features_cat),
        ("StandardScaler",numeric_transformer,features_num)
    ]
)

In [34]:
X_train = preprocessor.fit_transform(X_train)
X_test = preprocessor.transform(X_test)

Random Forest Training

In [39]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import accuracy_score,confusion_matrix,classification_report,f1_score,recall_score,precision_score

In [40]:
models = {
    'Random Forest':RandomForestClassifier(),
    'Decision Tree':DecisionTreeClassifier(),
    'Logistic Regression':LogisticRegression(),
    'Gradient Boosting':GradientBoostingClassifier(),
    'AdaBoost':AdaBoostClassifier()
}
for i in range(len(models)):
  model = list(models.values())[i]
  model.fit(X_train,y_train) ## Train Model

  ## Make Prediction
  y_train_pred = model.predict(X_train)
  y_test_pred = model.predict(X_test)

  ## Training set Transform
  model_train_accuracy = accuracy_score(y_train,y_train_pred)
  model_train_f1 = f1_score(y_train,y_train_pred,average='weighted')
  model_train_recall = recall_score(y_train,y_train_pred,average='weighted')
  model_train_precision = precision_score(y_train,y_train_pred,average='weighted')
  model_train_cm = confusion_matrix(y_train,y_train_pred)
  model_train_cr = classification_report(y_train,y_train_pred)

  ## Test set Transform
  model_test_accuracy = accuracy_score(y_test,y_test)
  model_test_f1 = f1_score(y_test,y_test_pred,average='weighted')
  model_test_recall = recall_score(y_test,y_test_pred)
  model_test_precision = precision_score(y_test,y_test_pred)
  model_test_cm = confusion_matrix(y_test,y_test_pred)
  model_test_cr = classification_report(y_test,y_test_pred)

  print(list(models.keys())[i])
  print('Model Performance for Training set')
  print("- Accuracy: {:.4f}".format(model_train_accuracy))
  print('- F1 Score: {:.4f}'.format(model_train_f1))
  print('- Recall: {:.4f}'.format(model_train_recall))
  print('- Precision: {:.4f}'.format(model_train_precision))
  print('----------------------------------')
  print('Model Performance for Test set')
  print('- Accuracy: {:.4f}'.format(model_test_accuracy))
  print('- F1 Score: {:.4f}'.format(model_test_f1))
  print('- Recall: {:.4f}'.format(model_test_recall))
  print('- Precision: {:.4f}'.format(model_test_precision))
  print('='*35)
  print('\n')




Random Forest
Model Performance for Training set
- Accuracy: 1.0000
- F1 Score: 1.0000
- Recall: 1.0000
- Precision: 1.0000
----------------------------------
Model Performance for Test set
- Accuracy: 1.0000
- F1 Score: 0.9067
- Recall: 0.5864
- Precision: 0.9655


Decision Tree
Model Performance for Training set
- Accuracy: 1.0000
- F1 Score: 1.0000
- Recall: 1.0000
- Precision: 1.0000
----------------------------------
Model Performance for Test set
- Accuracy: 1.0000
- F1 Score: 0.8984
- Recall: 0.6859
- Precision: 0.7798


Logistic Regression
Model Performance for Training set
- Accuracy: 0.8468
- F1 Score: 0.8214
- Recall: 0.8468
- Precision: 0.8305
----------------------------------
Model Performance for Test set
- Accuracy: 1.0000
- F1 Score: 0.8086
- Recall: 0.2984
- Precision: 0.6786


Gradient Boosting
Model Performance for Training set
- Accuracy: 0.8923
- F1 Score: 0.8796
- Recall: 0.8923
- Precision: 0.8911
----------------------------------
Model Performance for Test set

Hyper Parameter Tunning